In [2]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import tensorflow as tf
from PIL import Image
import glob
import os
import re
import multiprocessing

%matplotlib inline

In [3]:
filepath = "/data/asl_alphabet_{0}/"
cwd = os.getcwd()

In [4]:
num_classes = 26
batch_size = 117
train_size = 70200
val_size = 7800

# Data Exploration and Preprocessing

In [5]:
def get_imagepaths(type="train"):
    imagepaths = []

    for dir in glob.glob(cwd + filepath.format(type) + "*/"):
        for imagepath in glob.glob(dir + "*.jpg"):
            imagepaths.append(imagepath)
    imagepaths = np.array(imagepaths)
    np.random.shuffle(imagepaths)
    
    return imagepaths

In [6]:
def build_data(imagepaths):
    data = []
    for p in imagepaths:
        label = re.search(r"\/([A-Z])\/", p).group(0)[1]
        label = int(ord(label) - ord("A"))
        data.append((p, label))

    return np.array(data, dtype=object)

In [7]:
def train_val_split(imagepaths, val_amt):
    data = build_data(imagepaths)
    amt = data.shape[0]
    train = data[:int(amt*(1-val_amt))]
    val = data[int(amt*(1-val_amt)):]
    
    return train, val

In [8]:
def create_tfrecord(data, type="train"):
    np.random.shuffle(data)
    tfr_dir = "/data/tfrecords/"
    if not os.path.exists(cwd + tfr_dir):
        os.makedirs(cwd + tfr_dir)
    tfr_filename = "{0}.tfrecords".format(type)
    if os.path.isfile(cwd + tfr_dir + tfr_filename):
        os.remove(cwd + tfr_dir + tfr_filename)
    writer = tf.python_io.TFRecordWriter(cwd + tfr_dir + tfr_filename)
    
    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
    def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    
    for p in data:
        feature = {
            "image": _bytes_feature(tf.compat.as_bytes(p[0])),
            "label": _int64_feature(p[1])
        }

        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())
            
    print(tfr_filename + " Complete")
    writer.close()
    
    return cwd + tfr_dir + tfr_filename

In [9]:
def _parse(protos):
    features = {"image": tf.FixedLenFeature((), tf.string, default_value=""),
                "label": tf.FixedLenFeature((), tf.int64, default_value=0)}
    parsed_features = tf.parse_example(protos, features)
    
    def decode(image_files):
        return tf.image.decode_jpeg(image_files, channels=3)
    
    image_files = tf.map_fn(tf.read_file, parsed_features["image"])
    images = tf.map_fn(decode, image_files, dtype=tf.uint8)
    labels = parsed_features["label"]
    
    return images, labels

In [10]:
def conv_layer(input, size_in, size_out, k_size, name):
    with tf.variable_scope(name) as scope:
        weights = tf.get_variable("weights", shape=[k_size, k_size, size_in, size_out])
        biases = tf.get_variable("biases", shape=[size_out], initializer=tf.constant_initializer(0.0))
        conv = tf.nn.conv2d(input, weights, strides=[1,1,1,1], padding="SAME")
        conv = tf.nn.bias_add(conv, biases)
        act = tf.nn.relu(conv, scope.name)
        
        tf.summary.histogram("weights", weights)
        tf.summary.histogram("biases", biases)
        tf.summary.histogram("activations", act)
        
    return act

def batch_norm_layer(input, is_training, eps, name):
    with tf.variable_scope(name) as scope:
        gamma = tf.get_variable("gamma", shape=[input.shape[0]], initializer=tf.constant_initializer(1.0))
        beta = tf.get_variable("beta", shape=[input.shape[0]], initializer=tf.constant_initializer(0.0))
        mean, var = tf.nn.moments(input, [0,1,2])
        ema = tf.nn.ExponentialMovingAverage(decay=0.99)
        
        def mean_var_with_update():
            ema_apply_op = ema.apply([mean, var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(mean), tf.identity(var)
            
        mean, var = tf.cond(is_training, mean_var_with_update, lambda: (ema.average(mean), ema.average(var)))
        bn = tf.nn.batch_normalization(input, mean, var, gamma, beta, eps)
        
        tf.summary.histogram("gamma", gamma)
        tf.summary.histogram("beta", beta)
        
    return bn
        
def dropout_layer(input, dropout):
    do = tf.nn.dropout(input, dropout)
    
    return do

def maxpool_layer(input, name):
    return tf.nn.max_pool(input, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME", name=name)

def fc_layer(input, size_in, size_out, name):
    with tf.variable_scope(name) as scope:
        weights = tf.get_variable("weights", shape=[size_in, size_out])
        biases = tf.get_variable("biases", shape=[size_out])
        act = tf.matmul(input, weights) + biases
        
        tf.summary.histogram("weights", weights)
        tf.summary.histogram("biases", biases)
        tf.summary.histogram("activations", act)
        
    return act

In [11]:
def model(X, dropout):
    
    tf.summary.image("input", X, 10)

    conv1 = conv_layer(X, 3, 32, 3, "conv1")
    conv2 = conv_layer(conv1, 32, 32, 3, "conv2")
    maxpool1 = maxpool_layer(conv2, "maxpool1")
    conv3 = conv_layer(maxpool1, 32, 32, 3, "conv3")
    conv4 = conv_layer(conv3, 32, 32, 3, "conv4")
    maxpool2 = maxpool_layer(conv4, "maxpool2")
    conv5 = conv_layer(maxpool2, 32, 32, 3, "conv5")
    conv6 = conv_layer(conv5, 32, 32, 3, "conv6")
    maxpool3 = maxpool_layer(conv6, "maxpool3")
    flatten = tf.reshape(maxpool3, shape=[-1, tf.reduce_prod(maxpool3.shape[1:])])
    fc1 = fc_layer(flatten, 20000, 256, "fc1")
    dropout1 = dropout_layer(fc1, dropout)
    fc2 = fc_layer(dropout1, dropout1.shape[1], 256, "fc2")
    dropout2 = dropout_layer(fc2, dropout)
    fc3 = fc_layer(dropout2, dropout2.shape[1], 26, "fc3")

    return fc3

In [12]:
def loss(logits, labels):
    with tf.name_scope("loss"):
        labels = tf.cast(labels, tf.int64)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels)
        cross_entropy_mean = tf.reduce_mean(cross_entropy, name="cross_entropy")

    tf.summary.scalar("cross_entropy", cross_entropy_mean)
        
    return cross_entropy_mean

In [13]:
def training(loss, learning_rate):
    with tf.name_scope("training"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        train_op = optimizer.minimize(loss, global_step=global_step)

    return train_op

In [14]:
def evaluation(y_pred, y):
    correct = tf.equal(tf.argmax(tf.nn.softmax(y_pred), 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    tf.summary.scalar("accuracy", accuracy)
    
    return accuracy

In [15]:
imagepaths = get_imagepaths("train")
train, val = train_val_split(imagepaths, 0.1)
train_filepath = tf.placeholder(tf.string, name="train_filepath")
val_filepath = tf.placeholder(tf.string, name="val_filepath")

train_dataset = tf.data.TFRecordDataset(train_filepath)
train_dataset = train_dataset.shuffle(70200)
train_dataset = train_dataset.batch(117).map(_parse)
train_dataset = train_dataset.shuffle(117)
train_dataset = train_dataset.prefetch(2)

val_dataset = tf.data.TFRecordDataset(val_filepath)
val_dataset = val_dataset.shuffle(7800)
val_dataset = val_dataset.batch(100).map(_parse)
val_dataset = val_dataset.shuffle(100)
val_dataset = val_dataset.prefetch(2)

train_iter = train_dataset.make_initializable_iterator()
val_iter = val_dataset.make_initializable_iterator()
next_train = train_iter.get_next()
next_val = val_iter.get_next()

X = tf.placeholder(tf.float32, shape=[None, 200, 200, 3], name="X")
Y = tf.placeholder(tf.int64 , shape=[None, num_classes], name="Y")
keep_prob = tf.placeholder(tf.float32, name="keep_prob")
logits = model(X, keep_prob)
loss = loss(logits, Y)
train_op = training(loss, 0.0001)
accuracy = evaluation(logits, Y)

In [ ]:
with tf.Session() as sess:
    train_fp = create_tfrecord(train, "train")
    val_fp = create_tfrecord(val, "val")
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter(cwd + "/checkpoints/train", sess.graph)
    val_writer = tf.summary.FileWriter(cwd + "/checkpoints/val", sess.graph)
    summary = tf.summary.merge_all()
    
    for e in range(10):
        sess.run(train_iter.initializer, feed_dict={train_filepath: train_fp})
        train_l, train_acc = 0, 0
        pbar = tqdm(total=600)
        for step in range(600):
            images, labels = sess.run(next_train)
            labels = tf.keras.utils.to_categorical(labels, num_classes)
            sess.run(train_op, feed_dict={X: images, Y: labels, keep_prob: 0.5})
            l, acc, s = sess.run([loss, accuracy, summary], feed_dict={X: images, Y: labels, keep_prob: 1.0})
            train_writer.add_summary(s, e*600 + step)
            train_l += l * 117
            train_acc += acc * 117
            pbar.update(1)
            break
        pbar.close()
        
        sess.run(val_iter.initializer, feed_dict={val_filepath: val_fp})
        val_l, val_acc = 0, 0
        for step in range(78):
            images, labels = sess.run(next_val)
            labels = tf.keras.utils.to_categorical(labels, num_classes)
            l, acc, s = sess.run([loss, accuracy, summary], feed_dict={X: images, Y: labels, keep_prob: 1.0})
            val_writer.add_summary(s, e*600+step)
            val_l += l * 100
            val_acc += acc * 100
        print("Epoch: {0}, Training Loss: {1},  Training Accuracy: {2}. Validation Loss: {3}, Validation Accuracy: {4}"
              .format(e, train_l/600, train_acc/600, val_l/78, val_acc/78))
        

train.tfrecords Complete
val.tfrecords Complete


Epoch: 0, Training Loss: 2.7289855527877807,  Training Accuracy: 0.006666666995733976. Validation Loss: 1542.7453970297788, Validation Accuracy: 2.6923076512339787


Epoch: 1, Training Loss: 2.3905966472625733,  Training Accuracy: 0.008333333563059568. Validation Loss: 1293.9374349056145, Validation Accuracy: 3.4743589205810657


Epoch: 2, Training Loss: 1.9765664291381837,  Training Accuracy: 0.013333333991467952. Validation Loss: 1085.1797201694587, Validation Accuracy: 4.564102524175094


Epoch: 3, Training Loss: 1.5744908809661866,  Training Accuracy: 0.011666666697710753. Validation Loss: 871.3565153953357, Validation Accuracy: 3.9358973909073915
